https://pytorch.org/blog/finetune-llms/

https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing

# 1. Load Environment & Huggingface, W&B login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_04"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Dataset Load

- 이 시스템에서는 작업을 외장하드에서 하고 있기 때문에 캐쉬폴더를 별도로 지정(In this system, tasks are being performed on an external hard drive, so a separate cache folder is specified.)

In [3]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [4]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [5]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [6]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.5)

In [7]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 76315
    })
    test: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 76315
    })
})

In [8]:
# ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.9)

# 3. Model Load

- QLoRA로 finetune을 하기위해 일반 7B 모델을 BitsAndBytes를 통해 양자화(4bit quantization)하여 로드, 훈련속도 향상을 위해 데이터 타입을 fp32가 아니라 fp16으로 로드(차후에는 bf16으로 로드해서 훈련예정)
- To finetune with QLoRA, a standard 7B model is quantized (4-bit quantization) through BitsAndBytes for loading, and to improve training speed, the data type is loaded as fp16 instead of fp32 (with plans to load and train with bf16 in the future).


```python

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
```

### *Reference

- https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py
- https://colab.research.google.com/drive/19AFEOrCI6-bc7h9RTso_NndRwXJRaJ25?usp=sharing

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

In [10]:
checkpoint_dir = "./checkpoint/experi_04"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [11]:
# 4bit QLoRA 학습을 위한 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [13]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

In [14]:
# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

tokenizer.add_special_tokens({'pad_token': '<PAD>'})
# tokenizer.pad_token = tokenizer.eos_token

# 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
# tokenizer.padding_side = "right"

1

In [16]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [17]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/76315 [00:00<?, ? examples/s]

In [18]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 76315
})

In [19]:
train_dataset["text"][1000]

'### instruction: 동료나 선배가 회의에서 발언하려고 할 때마다 계속 방해합니다. 이런 상황에서는 어떻게 대처하시겠습니까?\n\n### output: 회의에서 발언하려고 할 때마다 동료나 선배가 계속 저를 방해한다면, 저는 다음 단계를 통해 상황을 처리할 것입니다:1. 침착하고 평정심을 유지합니다: 침착하고 평정심을 유지하는 것이 중요하며, 방해가 제 감정에 영향을 미치거나 회의의 목적에 방해가 되지 않도록 하는 것이 중요합니다.2. 정중하게 내 요점 주장하기: 정중하지만 단호하게 내 요점을 주장하고 상대방이 끼어들기 전에 내 생각을 마칠 수 있게 해달라고 요청합니다. "실례합니다, 제 말을 마무리해도 될까요?" 또는 "응답하기 전에 제 말을 끝내 주시면 감사하겠습니다."와 같은 문구를 사용합니다.3. 적극적으로 경청하는 모습을 보여주세요: 상대방의 요점을 인정하고 사려 깊은 답변을 제공함으로써 적극적으로 경청하는 모습을 보이는 것이 중요합니다. 이렇게 함으로써 내가 대화에 참여하고 있다는 것을 보여줄 수 있고 상대방도 나의 예의에 화답하도록 유도할 수 있습니다.4. 방해에 대처하기: 상대방이 계속해서 제 말을 끊는다면 "다른 관점을 가지고 있다는 것은 이해하지만, 제 생각을 마칠 수 있게 해 주시면 답변하는 데 도움이 될 것 같습니다."와 같이 직접 문제를 해결합니다.5. 그 사람과 개인적으로 이야기합니다: 위의 단계를 시도한 후에도 방해가 계속되는 경우, 회의가 끝난 후 상대방과 개인적으로 이야기하여 우려 사항을 공유합니다. 건설적인 방식으로 대화를 나누고 우리 모두에게 도움이 되는 해결책을 찾으려고 노력합니다.요약하자면, 회의 중에 동료나 선배가 계속 제 말을 끊는다면 침착함을 유지하고, 정중하게 내 주장을 주장하며, 적극적으로 경청하는 태도를 보이고, 방해 행위를 직접 해결하고, 필요한 경우 그 사람과 개인적으로 이야기합니다. 이러한 조치를 취함으로써 저는 제 아이디어가 전문적이고 건설적인 방식으로 경청되고 가치를 인정받을 수 있습니다.</

In [20]:
train_dataset["text"][3000]

'### instruction: 뉴욕에서 5일간 여행할 수 있는 프로그램을 계획하는 데 도움을 주실 수 있나요? 아이 3명, 부모님\n\n### output: 물론이죠! 가족을 위한 5일간의 뉴욕 여행을 계획하는 데 기꺼이 도와드리겠습니다. 어린이를 동반한 가족에게 적합한 여행 일정을 소개합니다:1일차:* 상징적인 자유의 여신상과 엘리스 아일랜드 방문으로 하루를 시작하세요. 사전에 티켓을 예약하세요.* 그 후 배터리 공원을 산책하며 해안가와 뉴욕 항구를 감상하세요. 근처에 있는 많은 포장마차나 레스토랑에서 점심을 드실 수도 있습니다.* 오후에는 국립 9/11 기념관 및 박물관으로 향하세요. 아이들에게 교육적인 경험이 될 수 있는 감동적인 체험입니다.* 유서 깊은 사우스 스트리트 항구를 산책하고 거리 공연자와 음악가들을 구경하며 하루를 마무리하세요.2일차:* 센트럴 파크 방문으로 하루를 시작하세요. 자전거를 빌리거나 공원을 여유롭게 산책할 수 있습니다. 센트럴 파크 동물원과 센트럴 파크 컨서버토리 가든도 꼭 들러보세요.* 오후에는 미국 자연사 박물관으로 향하세요. 이 박물관에는 공룡 화석, 대왕고래 등 3,000만 개가 넘는 표본이 전시되어 있습니다.* 타임스퀘어 방문으로 하루를 마무리하세요. 아이들은 밝은 조명과 거리 공연자, 그리고 이 지역의 전반적인 에너지를 즐길 수 있습니다.셋째 날:* 엠파이어 스테이트 빌딩 방문으로 하루를 시작하세요. 인파를 피하고 전망대에서 멋진 경치를 즐기려면 일찍 도착하세요.* 다음으로 인트레피드 바다, 항공 및 우주 박물관으로 향하세요. 이 박물관에는 역사적인 항공기, 우주선 및 해군 선박 컬렉션이 전시되어 있습니다.* 오후에는 브루클린 다리를 건너 도시 스카이라인의 장관을 감상하며 산책을 즐겨 보세요.* 브루클린의 윌리엄스버그 또는 덤보의 트렌디한 동네를 둘러보며 하루를 마무리하세요. 두 지역 모두 훌륭한 레스토랑과 상점이 많이 있습니다.4일차:* 메트로폴리탄 미술관 방문으로 하루를 시작하세요. 이 박물관은 전 세계의 방대한 

In [21]:
train_dataset["text"][3230]

'### instruction: 도커와 VM 머신의 차이점을 이해하지 못했나요?\n\n### output: Docker와 가상 머신(VM)은 모두 격리된 환경에서 애플리케이션을 실행하는 데 사용할 수 있는 도구입니다. 그러나 둘 사이에는 몇 가지 주요 차이점이 있습니다.Docker와 VM의 주요 차이점 중 하나는 VM은 하이퍼바이저에서 실행되는 반면, Docker 컨테이너는 호스트 운영 체제에서 실행된다는 점입니다. 즉, VM은 실행에 메모리 및 스토리지와 같은 더 많은 리소스가 필요한 반면, Docker 컨테이너는 더 가볍고 효율적입니다.또 다른 주요 차이점은 VM은 각 애플리케이션에 대해 완전한 독립형 운영 체제를 제공하는 반면, Docker 컨테이너는 호스트 운영 체제를 공유하며 애플리케이션을 실행하는 데 필요한 라이브러리와 종속성만 포함한다는 것입니다. 즉, Docker 컨테이너는 이식성이 뛰어나고 다른 환경 간에 쉽게 이동할 수 있는 반면, VM은 더 격리되어 있으며 특정 운영 체제나 구성이 필요한 애플리케이션에 더 적합합니다.전반적으로 Docker와 VM 중 어떤 것을 사용할지는 애플리케이션과 환경의 특정 요구사항에 따라 달라집니다. 휴대성과 가벼운 실행이 필요한 애플리케이션에는 Docker가 적합하며, 보다 격리되고 사용자 지정 가능한 환경이 필요한 애플리케이션에는 VM이 더 적합합니다.</s>'

In [22]:
train_dataset["text"][12312]

'### instruction: 미국 빈곤의 원인을 조사하는 논문에 대한 연구 개요를 작성합니다.\n\n### output: I. 서론\n    A. 빈곤의 정의\n    B. 미국의 빈곤에 대한 간략한 개요\n    C. 백서의 목적\n\nII. 미국의 빈곤에 대한 배경 정보\n    A. 빈곤에 관한 통계 데이터\n    B. 경제 역사 개요\n    C. 빈곤 문제를 해결하려는 정부 정책\n\nIII. 빈곤의 원인\n    A. 경제적 요인\n        1. 실업 및 불완전 고용\n        2. 소득 분배의 불평등\n        3. 교육 및 취업 기회에 대한 접근성 부족\n    B. 사회적 요인\n        1. 차별과 편견\n        2. 가족 구조\n        3. 건강 격차\n    C. 정부 정책\n        1. 부적절한 사회복지 프로그램\n        2. 세금 정책\n        3. 주택 정책\n    \nIV. 결론\n    가. 주요 내용 요약\n    나. 연구의 시사점\n    C. 향후 연구를 위한 제언.</s>'

In [23]:
# test_dataset = ko_instruction_01['test'].map(template_dataset, remove_columns=list(ko_instruction_01['test'].features), num_proc=10)

In [24]:
from transformers import TrainingArguments


output_dir = "./checkpoint/experi_04"
per_device_train_batch_size = 3
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 20
save_total_limit=10
logging_steps = 5
learning_rate = 2e-4
num_train_epochs=2
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    num_train_epochs = num_train_epochs,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

In [25]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    dataset_text_field="text",
    packing=True,
    tokenizer=tokenizer,
    max_seq_length=1024,
)

Generating train split: 0 examples [00:00, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
trainer.train()


  0%|          | 0/19176 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.8438, 'grad_norm': 0.8151676058769226, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 2.568, 'grad_norm': 0.5109564661979675, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 2.3867, 'grad_norm': 1.1732099056243896, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 2.1035, 'grad_norm': 0.33722248673439026, 'learning_rate': 0.0002, 'epoch': 0.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.9351, 'grad_norm': 0.306474506855011, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.9709, 'grad_norm': 0.24044185876846313, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.9751, 'grad_norm': 0.26627326011657715, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.8948, 'grad_norm': 0.3147087097167969, 'learning_rate': 0.0002, 'epoch': 0.0}


...

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.09}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


KeyboardInterrupt: 

### 허깅페이스에서 훈련 도중 러닝레이트가 수정 안되는 경우가 많다 한다. 
- 스케쥴러 변경을 시도해 보기로 한다. 

In [ ]:
output_dir = "./results/experi_04"

In [ ]:
trainer.model.save_pretrained(output_dir)